In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast

In [2]:
%pwd

'/Users/alisdghnia/Desktop/KNN-from-scratch'

In [39]:
train_df = pd.read_csv('spam_train.csv')
test_df = pd.read_csv('spam_test.csv')

In [40]:
train_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f49,f50,f51,f52,f53,f54,f55,f56,f57,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.250,0.0,0.000,0.000,0.000,1.601,8,173,0
2296,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.632,0.0,0.000,0.000,0.000,2.142,11,45,0
2297,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.091,0.0,0.045,0.000,0.000,2.333,15,175,0
2298,0.00,0.00,0.60,0.0,0.60,0.00,0.00,0.00,0.00,0.00,...,0.00,0.110,0.0,0.000,0.000,0.000,1.074,3,29,0


In [41]:
test_df = test_df.drop(' ID', axis = 1)

In [42]:
X_train, y_train, X_test, y_test = train_df.iloc[:, :-1], train_df.iloc[:, -1], test_df.iloc[:, :-1], test_df.iloc[:, -1]

In [60]:
def calculate_distances(train_data, test_point):
    distances = []
    for train_point in train_data:
        distance = np.sum((train_point[:-1] - test_point[:-1])**2)
        distances.append(np.sqrt(distance))
    return np.array(distances)

def knn_predict(distances, train_labels, k=3):
    indices = np.argsort(distances)[:k]
    k_nearest_labels = train_labels[indices]
    unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
    predicted_label = unique_labels[np.argmax(counts)]
    return predicted_label

def knn_classifier(train_data, train_labels, test_data, k_values):
    all_distances = []
    predictions_dict = {}

    for k in k_values:
        predictions = []
        for test_point in test_data:
            distances = calculate_distances(train_data, test_point)
            predictions.append(knn_predict(distances, train_labels, k))
        predictions_dict[k] = predictions
        all_distances.append(distances)

    return predictions_dict, all_distances


train_data = X_train.to_numpy()
train_labels = y_train.to_numpy()
test_data = X_test.to_numpy()

k_values = [1, 5, 11, 21, 41, 61, 81, 101, 201, 401]

predictions_dict, all_distances = knn_classifier(train_data, train_labels, test_data, k_values)

In [62]:
accuracies = []
for k in k_values:
    correct = 0
    for i in range(len(y_test)):
        if predictions_dict[k][i] == y_test[i]:
            correct += 1
    accuracies.append(correct / len(y_test))

accuracies

[0.8300738809213386,
 0.834419817470665,
 0.837896566710126,
 0.8426770969143851,
 0.8265971316818774,
 0.8209474141677532,
 0.8226857887874837,
 0.8105171664493699,
 0.7957409821816601,
 0.737070838765754]

In [63]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [66]:
predictions_dict_norm, all_distances_norm = knn_classifier(X_train_norm, train_labels, X_test_norm, k_values)

In [68]:
accuracies_norm = []
for k in k_values:
    correct = 0
    for i in range(len(y_test)):
        if predictions_dict_norm[k][i] == y_test[i]:
            correct += 1
    accuracies_norm.append(correct / len(y_test))

accuracies_norm

[0.8539765319426337,
 0.8700564971751412,
 0.878748370273794,
 0.8796175575836592,
 0.8839634941329857,
 0.8800521512385919,
 0.8744024337244676,
 0.8717948717948718,
 0.8600608431116906,
 0.8409387222946545]